In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_DEV=1

env: ANYWIDGET_DEV=1


In [2]:
import numpy as np
from seq.model import Event, EventGraph

In [3]:
data = np.random.randint(1, 5, (10_000, 32))
data.shape

(10000, 32)

In [4]:
G = EventGraph(data)

In [5]:
max_weight_edge = min(G.edges(data=True), key=lambda x: x[2]["weight"])

In [6]:
max_weight_edge

(Event(Position=17, Value=3, # Occurences=2510),
 Event(Position=18, Value=2, # Occurences=2531),
 {'weight': (1, 0.22071713147410357)})

In [7]:
from seq import Widget

w = Widget()
w

Widget()